In [1]:
pip install tensorflow==2.12

## Prerequisite

In [1]:
!pip install sklearn-evaluation

In [1]:
import sys
print(sys.version)


In [2]:
%pip install sklearn-evaluation


In [2]:
!pip install xlsxwriter

In [2]:
!pip install sklearn-evaluation


In [34]:
import numpy as np
import h5py
import tensorflow as tf
import pandas as pd
from pandas import DataFrame

# Data utilities
from pandas import read_csv
import matplotlib.pyplot as plt
# Data visualization
import plotly.express as px

# Machine learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import cross_val_score
from sklearn_evaluation import plot
from sklearn.metrics import accuracy_score, make_scorer, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, matthews_corrcoef, balanced_accuracy_score
from sklearn.metrics import accuracy_score, precision_recall_curve, auc
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score


In [4]:
from google.colab import drive
drive.mount('/content/drive')

In [35]:
import os
print(os.getcwd())


In [36]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido100.csv

In [3]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido80.csv

In [7]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido60.csv

In [8]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido40.csv

In [9]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido20.csv

## Utility Function

In [5]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [6]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)

  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [7]:
def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

In [8]:
def error_rate(testing_labels, predicted_testing_labels):
  from sklearn.metrics import f1_score, matthews_corrcoef, balanced_accuracy_score
  from sklearn.metrics import accuracy_score, precision_recall_curve
  from sklearn.metrics import classification_report
  from sklearn.metrics import recall_score, precision_score
  from sklearn.metrics import roc_curve, roc_auc_score, auc
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  f1_performances = list()
  pre = list()
  rec = list()
  bootstrap_auc_pr = list()
  bootstrap_auc_roc = list()

  Y = np.array(testing_labels) # convert list of groundtruths to numpy
  Yhat = np.array(predicted_testing_labels) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  metric = matthews_corrcoef # the metric you want to compute
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
    performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
    f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
    pre.append(precision_score(testing_labels[subset], predicted_testing_labels[subset]))
    rec.append(recall_score(testing_labels[subset], predicted_testing_labels[subset]))

    # Calculate ROC curve and AUC-ROC for the bootstrap sample
    fpr, tpr, thresholds = roc_curve(testing_labels[subset], predicted_testing_labels[subset])
    bootstrap_auc_roc.append(roc_auc_score(testing_labels[subset], predicted_testing_labels[subset]))

    # Calculate precision-recall curve and AUC-PR for the bootstrap sample
    precision, recall, thresholds = precision_recall_curve(testing_labels[subset], predicted_testing_labels[subset])
    bootstrap_auc_pr.append(auc(recall, precision))

  sd_mcc = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
  sd_acc = np.std(performances)
  sd_f1 = np.std(f1_performances)
  sd_pre = np.std(pre)
  sd_rec = np.std(rec)
  sd_auc_roc = np.std(bootstrap_auc_roc)
  sd_auc_pr = np.std(bootstrap_auc_pr)

  # Calculate ROC curve
  fpr, tpr, thresholds = roc_curve(Y, Yhat)
  auc_roc = roc_auc_score(Y, Yhat)

  # Plot the ROC curve
  plt.figure()
  plt.plot(fpr, tpr, label='ROC Curve')
  plt.xlabel('False Positive Rate (FPR)')
  plt.ylabel('True Positive Rate (TPR)')
  plt.title('ROC Curve')
  plt.grid(True)
  plt.xlim(0, 1)
  plt.ylim(0, 1)
  plt.legend(loc='lower right')
  plt.show()

  # Calculate precision-recall curve
  precision, recall, thresholds = precision_recall_curve(Y, Yhat)
  auc_pr = auc(recall, precision)

  # Plot ROC Curve

  plt.figure(figsize=(8, 6))
  plt.plot(fpr, tpr, label='ROC Curve (area = %0.2f)' % auc(fpr, tpr))
  plt.plot([0, 1], [0, 1], 'k--', label='reference point')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate (FPR)')
  plt.ylabel('True Positive Rate (TPR)')
  plt.title('ROC Curve')
  plt.legend(loc="lower right")
  plt.show()

  # Plot Precision-Recall Curve
  plt.figure(figsize=(8, 6))
  plt.step(recall, precision, where='post', label='Precision-Recall curve (area = %0.2f)' % auc_pr)
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.ylim([0.0, 1.05])
  plt.title('Precision-Recall Curve')
  plt.legend(loc="lower right")
  plt.show()

  print('acc:',accuracy_score(testing_labels, predicted_testing_labels))
  print('f1:',f1_score(testing_labels, predicted_testing_labels))
  print('mcc:',matthews_corrcoef(testing_labels, predicted_testing_labels))
  print('precision:',precision_score(testing_labels, np.round(Yhat)))
  print('recall:',recall_score(testing_labels, np.round(Yhat)))
  print('AUC-PR:', auc_pr)
  print('AUC-ROC:',auc_roc)
  print('sd_acc:',sd_acc)
  print('sd_f1:',sd_f1)
  print('sd_mcc:',sd_mcc)
  print('sd_pre:',sd_pre)
  print('sd_rec:',sd_rec)
  print('sd_auc_roc:',sd_auc_roc)
  print('sd_auc_pr:',sd_auc_pr)
  print(classification_report(testing_labels, predicted_testing_labels))

  return (sd_acc, sd_mcc, sd_f1)

In [9]:
def conf_matrix(confusion_matrix_data):
  from mlxtend.plotting import plot_confusion_matrix
  fig, ax = plot_confusion_matrix(conf_mat =confusion_matrix_data,
                                show_absolute=True,
                                show_normed=True,
                                #display_labels=class_dict.values(),
                                colorbar=True)
  labels = ['Non-acidophilic', 'Acidophilic']
  ax.set_xticklabels([''] + labels)
  ax.set_yticklabels([''] + labels)
  plt.show()


In [10]:
def mcc(clf,X,y):
  y_pred = clf.predict(X)
  mcc = matthews_corrcoef(y, y_pred)
  return mcc

In [11]:
def std_acc(clf,X,y):
  from sklearn.metrics import accuracy_score
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append( accuracy_score(y[subset], y_pred[subset]) )
  sd_acc = np.std(bootstrap_performances)*1.96
  return sd_acc

In [12]:
def std_f1(clf,X,y):
  from sklearn.metrics import f1_score
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append(f1_score(y[subset], y_pred[subset]) )
  sd_f1 = np.std(bootstrap_performances)*1.96
  return sd_f1

In [13]:
def std_mcc(clf,X,y):
  from sklearn.metrics import matthews_corrcoef
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append(matthews_corrcoef(y[subset], y_pred[subset]) )
  sd_mcc = np.std(bootstrap_performances)*1.96
  return sd_mcc

## Open embedding file

In [14]:
proteins = []

In [15]:
!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/Dataset/acidoSplit.csv
#!wget https://raw.githubusercontent.com/33220311/Acidophilic/main/acido100.csv

In [21]:
!wget https://github.com/33220311/Extremophilic/raw/main/Embeddings/acidoAdd.h5

In [22]:
with h5py.File('acidoAdd.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [23]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAdd2.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [24]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAdd3.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [25]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAdd4.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [26]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAddT5.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [27]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAddT52.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [28]:
with h5py.File('/content/drive/MyDrive/Acidophilic/acidoAddT53.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [29]:
#proteins

In [17]:
# Basic Protocol 3 — Step 5
annotations = read_csv('acidoSplit.csv')

In [18]:
annotations[:3]

In [19]:
# Basic Protocol 3 — Step 6
train_set = annotations[annotations.set == "train"]
test_set = annotations[annotations.set == "test"]

In [20]:
print(f"The train set contains {len(train_set)} samples, and we will test on {len(test_set)} samples.")

In [21]:
# Basic Protocol 3 — Step 7

training_embeddings = list()
training_identifiers = train_set.identifier.values
training_labels = train_set.label.values
print(len(training_identifiers))
print(len(training_labels))

In [22]:
train_set.identifier.values

In [23]:
testing_embeddings = list()
testing_identifiers = test_set.identifier.values
testing_labels = test_set.label.values
print(len(testing_identifiers))
print(len(testing_labels))

In [24]:
seq = dict(proteins)
delete = list()

In [25]:
for identifier in training_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            training_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [26]:
for identifier in testing_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            testing_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [27]:
np.where(training_identifiers=='GAX78753')

In [28]:
training_identifiers = np.delete(training_identifiers,985)
training_labels = np.delete(training_labels,985)

In [29]:
# A sanity check: make sure that the numbers are equal!
assert(len(training_identifiers) == len(training_embeddings))
assert(len(testing_identifiers) == len(testing_embeddings))

In [30]:
training_embeddings[:3]

In [32]:
%pip install openpyxl

In [33]:
df =pd.DataFrame(delete)
df.to_excel('/content/delete.xlsx')

In [45]:
delete

## Training

In [46]:
arr_train = np.array(training_embeddings)
nsample, nx, ny = arr_train.shape
train_dataset = arr_train.reshape((nsample, nx*ny))
train_dataset.shape

In [47]:
arr_test = np.array(testing_embeddings)
nsample, nx, ny = arr_test.shape
test_dataset = arr_test.reshape((nsample, nx*ny))
test_dataset.shape

### LR

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score

In [ ]:
lr = LogisticRegression()
lr_history = lr.fit(train_dataset, training_labels)
lr.score(test_dataset,testing_labels)

In [ ]:
scoring ={'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
scores = cross_validate(lr, train_dataset, training_labels, scoring=scoring, cv=10)
scores

In [ ]:
predicted_training_labels = lr.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

In [ ]:
predicted_testing_labels = lr.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

In [ ]:
error_rate(testing_labels, predicted_testing_labels)

In [ ]:
df =pd.DataFrame(train_dataset)
df.to_excel('/content/T5_train_dataset.xlsx')
df =pd.DataFrame(training_labels)
df.to_excel('/content/T5_training_labels.xlsx')
df =pd.DataFrame(predicted_training_labels)
df.to_excel('/content/T5_pred_train_labels.xlsx')

df =pd.DataFrame(test_dataset)
df.to_excel('/content/T5_test_dataset.xlsx')
df =pd.DataFrame(testing_labels)
df.to_excel('/content/T5_testing_labels.xlsx')
df =pd.DataFrame(predicted_testing_labels)
df.to_excel('/content/T5_pred_test_labels.xlsx')

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)
conf_matrix(confusion_matrix_data)

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [ ]:
sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
#lr.save_weights('/result/LR.h5')
print(lr.coef_.shape)
lr_weights = lr.coef_
print(lr_weights)
print(np.max(lr_weights))
print(np.min(lr_weights))

### MLP with Grid Search

In [ ]:
# Machine learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Basic Protocol 3 — Step 8

multilayerperceptron = MLPClassifier(solver='lbfgs', random_state=10, max_iter=1000)

parameters = {
    'hidden_layer_sizes': [(2560,)],
    'learning_rate_init': [ 0.001],
    'solver':['sgd'],
    'learning_rate': ['constant'],
}

In [ ]:
grid_scorer ={'accuracy':make_scorer(accuracy_score),'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
# Basic Protocol 3 — Step 9

classifiers = GridSearchCV(multilayerperceptron, parameters, cv=10, scoring=grid_scorer, refit='mcc')
history = classifiers.fit(train_dataset, training_labels)
classifier = classifiers.best_estimator_

In [ ]:
df =pd.DataFrame(classifiers.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5BMLP61.xlsx')

In [ ]:
#plot.grid_search(classifiers.cv_results_, change= 'hidden_layer_sizes', kind='bar')

In [ ]:
params = classifier.get_params()
params

In [ ]:
predicted_training_labels = classifier.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

In [ ]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

In [ ]:
error_rate(testing_labels, predicted_testing_labels)

In [ ]:
#cv_results = DataFrame(classifiers.cv_results_)

#cv_results[['param_hidden_layer_sizes','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)
conf_matrix(confusion_matrix_data)

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [ ]:
sn = TP / float(TP + FN)
print(sn)

In [ ]:
sp = TN / float(TN + FP)
print(sp)

### CNN with GridSearch

In [ ]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from time import time

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [ ]:
n_epochs = 30 # 30
n_epochs_cv = 10 # 10  # reduce number of epochs for cross validation for performance reason

n_cv = 3
validation_ratio = 0.10

In [ ]:
def create_cnn_model(pool_type='max', conv_activation='tanh', dropout_rate=0.1, kernel=3, optimizer='adam'):
    # create model
    model = Sequential()

    # first layer: convolution
    #model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv1D(filters=128, kernel_size=kernel, activation=conv_activation,input_shape=(n_features,1)))
    # second series of layers: convolution, pooling, and dropout
    model.add(Conv1D(32, kernel_size=kernel, activation=conv_activation))
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    # third series of layers: convolution, pooling, and dropout
    model.add(Conv1D(64, kernel_size=kernel, activation=conv_activation))   # 32   10
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    #model.add(Conv1D(64, kernel_size=kernel, activation=conv_activation))   # 32   10
    #if pool_type == 'max':
    #    model.add(MaxPooling1D(pool_size=2))
    #if pool_type == 'average':
    #    model.add(AveragePooling1D(pool_size=2))
    #if dropout_rate != 0:
    #    model.add(Dropout(rate=dropout_rate))

    # fourth series
    model.add(Flatten())
    model.add(Dense(64, activation='sigmoid')) # 64
    # add a dropout layer if rate is not null
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer= optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'],
        )
    return model

In [ ]:
cnn = create_cnn_model()

In [ ]:
cnn.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
cnn.summary()

In [ ]:
# optimize model
start = time()

# create model
model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
# define parameters and values for grid search
param_grid = {
    'pool_type': ['max'], #max
    'conv_activation': ['relu'],#relu
    'epochs': [10],
    'kernel':[2], #2
    'optimizer':['adam'] #adam
}

In [ ]:
grid_scorer = {'accuracy':make_scorer(accuracy_score),'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=10, scoring=grid_scorer, refit='mcc')
grid_result = grid.fit(train_dataset, Y_train)

In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

In [ ]:
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))

In [ ]:
# summarize results
#print('time for grid search = {:.0f} sec'.format(time()-start))
#display_cv_results(grid_result)

In [ ]:
#DataFrame(grid.cv_results_)

In [ ]:
predicted_training_labels = classifier.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

In [ ]:
df =pd.DataFrame(grid.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5CNN60.xlsx')

In [ ]:
classifier = grid.best_estimator_

params = classifier.get_params()
params

In [ ]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

In [ ]:
error_rate(testing_labels, predicted_testing_labels)

In [ ]:
#from pandas import DataFrame
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_conv_activation','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)
conf_matrix(confusion_matrix_data)

In [ ]:
#TP = confusion_matrix_data[1,1]
#TN = confusion_matrix_data[0,0]
#FP = confusion_matrix_data[0,1]
#FN = confusion_matrix_data[1,0]

In [ ]:
#sn = TP / float(TP + FN)
#print(sn)

In [ ]:
#sp = TN / float(TN + FP)
#print(sp)

### RNN with GridSearch

In [49]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer, GRU
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
max_length = 1280

In [ ]:
def create_rnn_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    rnn = SimpleRNN(units, activity_regularizer=l2(regularizer), return_sequences = True)(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(rnn)
    rnn2 =SimpleRNN(units, activity_regularizer=l2(dropout_rate), return_sequences = True)(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(rnn2)
    rnn3 =SimpleRNN(units, activity_regularizer=l2(dropout_rate))(x2) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x3 = Dropout(dropout_rate)(rnn3)
    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x3)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),#solver,#
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
rnn_model = create_rnn_model()
rnn_model.summary()

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_rnn_model, verbose=1, epochs=5, batch_size=256)
# define parameters and values for grid search

parameters = {
    'units':[50], # 1024
    'learning_rate_init': [0.0003],
    #'solver':['adam'],
    'epochs':[5],
    'dropout_rate':[0.0], #0.05
    'regularizer':[0.0],
}

In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5RNN5.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

In [ ]:
predicted_rnn = classifier.predict(X_test)
predicted_rnn = np.where(predicted_rnn > 0.5, 1, 0)
predicted_rnn = np.reshape(predicted_rnn,(len(predicted_rnn),)).astype(int)
error_rate(Y_test, predicted_rnn)

In [ ]:
from sklearn.metrics import confusion_matrix

classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, predicted_rnn, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data

### GRU with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer, GRU
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
max_length = 1280

In [ ]:
def create_gru_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    gru = GRU(units, activity_regularizer=l2(regularizer), return_sequences = True)(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(gru)
    gru2 =GRU(units, activity_regularizer=l2(dropout_rate), return_sequences = True)(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(gru2)
    gru3 =GRU(units, activity_regularizer=l2(dropout_rate))(x2) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x3 = Dropout(dropout_rate)(gru3)
    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x3)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
gru_model = create_gru_model()
gru_model.summary()

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_gru_model, verbose=1, epochs=3, batch_size=128)
# define parameters and values for grid search

parameters = {
    'units':[128], # 1024
    'learning_rate_init': [1e-08],
    #'solver':['adam','sgd'],
    'epochs':[5],
    'dropout_rate':[0.0], #0.05
    'regularizer':[0.0],
}

In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5GRU3.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

In [ ]:
predicted_gru = classifier.predict(X_test)
predicted_gru = np.where(predicted_gru > 0.5, 1, 0)
predicted_gru = np.reshape(predicted_gru,(len(predicted_gru),)).astype(int)
error_rate(Y_test, predicted_gru)

In [ ]:
from sklearn.metrics import confusion_matrix

classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, predicted_gru, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data

### LSTM with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
def create_lstm_model(activation='tanh', dropout_rate=0.05, units=1024, learning_rate_init= 0.001):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    lstm = LSTM(units, activity_regularizer=l2(dropout_rate))(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    #x = Dropout(dropout_rate)(lstm)

    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(lstm)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
lstm_model = create_lstm_model()
lstm_model.summary()

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_lstm_model, verbose=1, epochs=3, batch_size=128)
# define parameters and values for grid search

parameters = {
    'units':[50], # 1024
    'learning_rate_init': [1e-06],
    #'solver':['adam','sgd'],
    'epochs':[5],
    'dropout_rate':[0.0], #0.05
    #'regularizer':[0.0],
}

In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5LSTM4.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

In [ ]:
predicted_lstm = classifier.predict(X_test)
predicted_lstm = np.where(predicted_lstm > 0.5, 1, 0)
predicted_lstm = np.reshape(predicted_lstm,(len(predicted_lstm),)).astype(int)
error_rate(Y_test, predicted_lstm)

In [ ]:
from sklearn.metrics import confusion_matrix

classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, predicted_lstm, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data

### BiLSTM with GridSearch

In [50]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time

In [51]:
from numpy.random import seed
seed(1)

In [52]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [53]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [54]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [55]:
input_shape = X_train[1,:].shape

In [56]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [57]:
num_words = 22
num_classes = 1
n_cv = 3
num_hiddens = 1024
num_steps = 10
num_layers = 1

In [58]:
def create_blstm_model1(activation='relu', dropout_rate=0.0, units=1024, learning_rate_init=0.001):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    bi_rnn = Bidirectional(LSTM(units, activation=activation, activity_regularizer=l2(dropout_rate),return_sequences=True))(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(bi_rnn)
    bi_rnn2 = Bidirectional(LSTM(units, activity_regularizer=l2(dropout_rate)))(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(bi_rnn2)

    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x2)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [59]:
blstm1 = create_blstm_model1()

In [60]:
blstm1.summary()

In [61]:
start = time()

# create model
model = KerasClassifier(build_fn=create_blstm_model1, verbose=1, epochs=3, batch_size=128)
# define parameters and values for grid search

parameters = {
    'units':[128], # 1024
    'learning_rate_init': [1e-8],
    #'activation':['relu'],
    #'solver':['adam','sgd'],
    'epochs':[10],
    'dropout_rate':[0.0], #0.05
    #'regularizer':[0.0],
}

In [62]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [63]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

In [64]:
classifier = grid.best_estimator_

In [65]:
params = classifier.get_params()
params

In [66]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results

In [67]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Acidophilic/T5BLSTM2.xlsx')

In [68]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

In [69]:
predicted_blstm = classifier.predict(X_test)
predicted_blstm = np.where(predicted_blstm > 0.5, 1, 0)
predicted_blstm = np.reshape(predicted_blstm,(len(predicted_blstm),)).astype(int)
error_rate(Y_test, predicted_blstm)

In [70]:
fpr_BLSTM , tpr_BLSTM, thresholds_BLSTM = roc_curve(Y_test, predicted_blstm)
auc_score = roc_auc_score(Y_test, predicted_blstm)
plt.plot(fpr_BLSTM, tpr_BLSTM, label= "BLSTM ({:.2f})".format(auc_score))

In [71]:
from sklearn.metrics import confusion_matrix

classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, predicted_blstm, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data